In [ ]:
import xarray as xr
import numpy as np

import matplotlib.pyplot as plt
import numpy as np

import cartopy.crs as ccrs
import cartopy.feature as cfeature

import statsmodels.api as sm 

In [ ]:
# RUN FOR HISTORICAL

file_dir = '[HISTORICAL FILE DIRECTORY]'

## use open_mfdataset to open multiple files at the same time
# radiative fluxes 

SWd = (xr.open_mfdataset(file_dir+'rsds_MPI-ESM1-2-HR_historical1850-2014_multi-realization.nc').rsds)
SWu = (xr.open_mfdataset(file_dir+'rsus_MPI-ESM1-2-HR_historical1850-2014_multi-realization.nc').rsus)
LWd = (xr.open_mfdataset(file_dir+'rlds_MPI-ESM1-2-HR_historical1850-2014_multi-realization.nc').rlds)
LWu = (xr.open_mfdataset(file_dir+'rlus_MPI-ESM1-2-HR_historical1850-2014_multi-realization.nc').rlus)

# latent and sensible heat 
LH = (xr.open_mfdataset(file_dir+'hfls_MPI-ESM1-2-HR_historical1850-2014_multi-realization.nc').hfls)
SH = (xr.open_mfdataset(file_dir+'hfss_MPI-ESM1-2-HR_historical1850-2014_multi-realization.nc').hfss)
SWd_clr = (xr.open_mfdataset(file_dir+'rsdscs_MPI-ESM1-2-HR_historical1850-2014_multi-realization.nc').rsdscs)
LWd_clr = (xr.open_mfdataset(file_dir+'rldscs_MPI-ESM1-2-HR_historical1850-2014_multi-realization.nc').rldscs)

# surface air temperature  
tas = (xr.open_mfdataset(file_dir+'ts_MPI-ESM1-2-HR_historical1850-2014_multi-realization.nc').ts)

# dimensions of data
lat = tas.lat
lon = tas.lon
ny = len(lat)
nx = len(lon)
year = tas.year

# heat storage
Q = SWd - SWu + LWd - LWu - LH - SH

# albedo (surface)
alb = SWu/SWd
SWd_cld = SWd - SWd_clr
LWd_cld = LWd - LWd_clr

# climatologies
alb_m = alb.mean(dim='year')
tas_m = tas.mean(dim='year')

tas_m_const = tas.mean(dim='year')

c = 5.670373*1.0e-8

# for use in loop:

SWduse = SWd
SWuuse = SWu
LWduse = LWd
LWuuse = LWu

LHuse = LH
SHuse = SH
SWd_clruse = SWd_clr
LWd_clruse = LWd_clr

tasuse = tas

Quse = Q

albuse = alb
SWd_clduse = SWd_cld
LWd_clduse = LWd_cld

alb_muse = alb_m
tas_muse = tas_m

# Calc partial temp changes (PTCs)
def delta(var):
    # Reshape the data to create a 2D array
    data = np.stack([var.values, amoc], axis=-1)
    
    # Perform the polynomial fit along the last dimension (0: var, 1: amoc)
    polyfit_coeffs = np.polyfit(-data[:, 1], data[:, 0], deg=1)
    
    # Return the coefficient for the linear term (slope)
    return polyfit_coeffs[0]

# Bar Plot -- Calculating Regional Mean for HISTORICAL ################################################################################################################

def nawh(var):
    # to calculate North Atlantic Warming Hole regional [44W-24W, 48N-60N] mean
    # migth re-define NAWH region 
    # var: an xarray [...,lat,lon]
    if (min(lon) < -100): # lon: -180~180
        x_center = -26.5
    else: # lon: 0~360
        x_center = 333.5
    x_range = 11.5
    #gen = 11.25
    y_center = 54.75
    y_range = 6.25
    #gen = 8.75
    return var.where(abs(var.lat - y_center) <= y_range).where(abs(var.lon - x_center ) <= x_range).mean(('lat', 'lon'))

amoc = 0
def ols_conf(y,x=amoc):
    # y is some 1-D variable (xarray)
    # x is the time/year
    x = amoc
    mod = sm.OLS(y.values, sm.add_constant(x)).fit()
    return mod.conf_int(0.1)[1,0]-mod.params[1]

con_int = np.zeros(7)
term_nawh = np.zeros(7)

hist_term_storage = np.zeros((10,7))
hist_con_storage = np.zeros((10,7))

term_name = ['SAF','CRF','non-SAF ClrSky SWR','ClrSky LWR↓','heat storage','SH&LH', 't5 + t6']

########## calculate regional mean PTC 

for r in range(0,10):

    amoc = xr.open_dataset('/Volumes/HDDBackup/CMIP6/Updated_AMOCI/AMOCI_historical/AMOCI_historical_multi_rlzn.nc')
    amoc = amoc.msftmz[r,:]
    amoc = amoc.values / 1e9

    print("current rlzn = {}".format(r+1))
    #assign current rlzn
    const=4*c*nawh(tas_m_const[r])**3
    SWd = SWduse[r]
    SWu = SWuuse[r]
    LWd = LWduse[r]
    LWu = LWuuse[r]

    LH = LHuse[r]
    SH = SHuse[r]
    SWd_clr = SWd_clruse[r]
    LWd_clr = LWd_clruse[r]

    tas = tasuse[r]

    Q = Quse[r]

    alb = albuse[r]
    SWd_cld = SWd_clduse[r]
    LWd_cld = LWd_clduse[r]

    alb_m = alb_muse[r]
    tas_m = tas_muse[r]

    # surface albedo feedback 
    term_nawh[0] = -delta(nawh(alb))*(nawh(SWd.mean(dim='year'))+delta(nawh(SWd)))/const
    con_int[0] = - ols_conf(nawh(alb))*(nawh(SWd.mean(dim='year')))/const # neglect the second higher-order term
    print('done 1')

    # CRF: cloud radiative forcing term 
    term_nawh[1] = (delta(nawh(LWd_cld))+(1-nawh(alb_m))*delta(nawh(SWd_cld)))/const
    con_int[1] = (ols_conf(nawh(LWd_cld))+(1-nawh(alb_m))*ols_conf(nawh(SWd_cld)))/const
    print('done 2')

    # non-SAF-induced change in clear-sky SW
    term_nawh[2] = (1-nawh(alb_m))*delta(nawh(SWd_clr))/const
    con_int[2] = (1-nawh(alb_m))*ols_conf(nawh(SWd_clr))/const
    print('done 3')

    # downward clear-sky LW change
    term_nawh[3] = delta(nawh(LWd_clr))/const
    con_int[3] = ols_conf(nawh(LWd_clr))/const
    print('done 4')

    # heat storage change
    term_nawh[4] = -delta(nawh(Q))/const
    con_int[4] = -ols_conf(nawh(Q))/const
    print('done 5')

    # surface sensible & latent heat change
    term_nawh[5] = -delta(nawh(LH+SH))/const
    con_int[5] = -ols_conf(nawh(LH+SH))/const
    print('done 6')

    term_nawh[6] = term_nawh[5] + term_nawh[4]
    con_int[6] = 0
    print('done 7')

    hist_term_storage[r] = term_nawh
    hist_con_storage[r] = con_int


In [ ]:
# RUN FOR SSP245

file_dir = '[SSP FILE PATH]'

## use open_mfdataset to open multiple files at the same time
# radiative fluxes 

SWd = (xr.open_mfdataset(file_dir+'rsds_MPI-ESM1-2-HR_ssp2451850-2014_multi-realization.nc').rsds)
SWu = (xr.open_mfdataset(file_dir+'rsus_MPI-ESM1-2-HR_ssp2451850-2014_multi-realization.nc').rsus)
LWd = (xr.open_mfdataset(file_dir+'rlds_MPI-ESM1-2-HR_ssp2451850-2014_multi-realization.nc').rlds)
LWu = (xr.open_mfdataset(file_dir+'rlus_MPI-ESM1-2-HR_ssp2451850-2014_multi-realization.nc').rlus)

# latent and sensible heat 
LH = (xr.open_mfdataset(file_dir+'hfls_MPI-ESM1-2-HR_ssp2451850-2014_multi-realization.nc').hfls)
SH = (xr.open_mfdataset(file_dir+'hfss_MPI-ESM1-2-HR_ssp2451850-2014_multi-realization.nc').hfss)
SWd_clr = (xr.open_mfdataset(file_dir+'rsdscs_MPI-ESM1-2-HR_ssp2451850-2014_multi-realization.nc').rsdscs)
LWd_clr = (xr.open_mfdataset(file_dir+'rldscs_MPI-ESM1-2-HR_ssp2451850-2014_multi-realization.nc').rldscs)

# surface air temperature  
tas = (xr.open_mfdataset(file_dir+'ts_MPI-ESM1-2-HR_ssp2451850-2014_multi-realization.nc').ts)

# dimensions of data
lat = tas.lat
lon = tas.lon
ny = len(lat)
nx = len(lon)
year = tas.year

# heat storage
Q = SWd - SWu + LWd - LWu - LH - SH

# albedo (surface)
alb = SWu/SWd
SWd_cld = SWd - SWd_clr
LWd_cld = LWd - LWd_clr

# climatologies
alb_m = alb.mean(dim='year')
tas_m = tas.mean(dim='year')

tas_m_const = tas.mean(dim='year')

c = 5.670373*1.0e-8

# for use in loop:

SWduse = SWd
SWuuse = SWu
LWduse = LWd
LWuuse = LWu

LHuse = LH
SHuse = SH
SWd_clruse = SWd_clr
LWd_clruse = LWd_clr

tasuse = tas

Quse = Q

albuse = alb
SWd_clduse = SWd_cld
LWd_clduse = LWd_cld

alb_muse = alb_m
tas_muse = tas_m

# Calc partial temp changes (PTCs)
def delta(var):
    # Reshape the data to create a 2D array
    var = var[:-1]
    data = np.stack([var.values, amoc], axis=-1)
    
    # Perform the polynomial fit along the last dimension (0: var, 1: amoc)
    polyfit_coeffs = np.polyfit(-data[:, 1], data[:, 0], deg=1)
    
    # Return the coefficient for the linear term (slope)
    return polyfit_coeffs[0]

# Bar Plot -- Calculating Regional Mean for SSP245 ################################################################################################################

def nawh(var):
    # to calculate North Atlantic Warming Hole regional [44W-24W, 48N-60N] mean
    # migth re-define NAWH region 
    # var: an xarray [...,lat,lon]
    if (min(lon) < -100): # lon: -180~180
        x_center = -26.5
    else: # lon: 0~360
        x_center = 333.5
    x_range = 11.5
    #gen = 11.25
    y_center = 54.75
    y_range = 6.25
    #gen = 8.75
    return var.where(abs(var.lat - y_center) <= y_range).where(abs(var.lon - x_center ) <= x_range).mean(('lat', 'lon'))

def ols_conf(y,x=amoc):
    # y is some 1-D variable (xarray)
    # x is the time/year
    x = amoc
    y = y[:-1]
    mod = sm.OLS(y.values, sm.add_constant(x)).fit()
    return mod.conf_int(0.1)[1,0]-mod.params[1]

con_int = np.zeros(7)
term_nawh = np.zeros(7)

ssp_term_storage = np.zeros((2,7))
ssp_con_storage = np.zeros((2,7))

term_name = ['SAF','CRF','non-SAF ClrSky SWR','ClrSky LWR↓','heat storage','SH&LH', 't5 + t6']

########## calculate regional mean PTC 

for r in range(0,2):

    amoc = xr.open_dataset('/Volumes/HDDBackup/CMIP6/Updated_AMOCI/AMOCI_ssp/AMOCI_ssp_multi_rlzn.nc')
    amoc = amoc.msftmz[r,:]
    amoc = amoc.values / 1e9

    print("current rlzn = {}".format(r+1))
    #assign current rlzn
    const=4*c*nawh(tas_m_const[r])**3
    SWd = SWduse[r]
    SWu = SWuuse[r]
    LWd = LWduse[r]
    LWu = LWuuse[r]

    LH = LHuse[r]
    SH = SHuse[r]
    SWd_clr = SWd_clruse[r]
    LWd_clr = LWd_clruse[r]

    tas = tasuse[r]

    Q = Quse[r]

    alb = albuse[r]
    SWd_cld = SWd_clduse[r]
    LWd_cld = LWd_clduse[r]

    alb_m = alb_muse[r]
    tas_m = tas_muse[r]

    # surface albedo feedback 
    term_nawh[0] = -delta(nawh(alb))*(nawh(SWd.mean(dim='year'))+delta(nawh(SWd)))/const
    con_int[0] = -ols_conf(nawh(alb))*(nawh(SWd.mean(dim='year')))/const # neglect the second higher-order term
    print('done 1')

    # CRF: cloud radiative forcing term 
    term_nawh[1] = (delta(nawh(LWd_cld))+(1-nawh(alb_m))*delta(nawh(SWd_cld)))/const
    con_int[1] = (ols_conf(nawh(LWd_cld))+(1-nawh(alb_m))*ols_conf(nawh(SWd_cld)))/const
    print('done 2')

    # non-SAF-induced change in clear-sky SW
    term_nawh[2] = (1-nawh(alb_m))*delta(nawh(SWd_clr))/const
    con_int[2] = (1-nawh(alb_m))*ols_conf(nawh(SWd_clr))/const
    print('done 3')

    # downward clear-sky LW change
    term_nawh[3] = delta(nawh(LWd_clr))/const
    con_int[3] = ols_conf(nawh(LWd_clr))/const
    print('done 4')

    # heat storage change
    term_nawh[4] = -delta(nawh(Q))/const
    con_int[4] = -ols_conf(nawh(Q))/const
    print('done 5')

    # surface sensible & latent heat change
    term_nawh[5] = -delta(nawh(LH+SH))/const
    con_int[5] = -ols_conf(nawh(LH+SH))/const
    print('done 6')

    term_nawh[6] = term_nawh[5] + term_nawh[4]
    con_int[6] = 0
    print('done 7')

    ssp_term_storage[r] = term_nawh
    ssp_con_storage[r] = con_int

In [ ]:
# RUN FOR Abrupt R1

file_dir = '[ABRUPT FILE PATH]'

## use open_mfdataset to open multiple files at the same time
# radiative fluxes 

SWd = (xr.open_mfdataset(file_dir+'rsds_MPI-ESM1-2-HR_abrupt-4xCO21850-2014_multi-realization.nc').rsds)
SWu = (xr.open_mfdataset(file_dir+'rsus_MPI-ESM1-2-HR_abrupt-4xCO21850-2014_multi-realization.nc').rsus)
LWd = (xr.open_mfdataset(file_dir+'rlds_MPI-ESM1-2-HR_abrupt-4xCO21850-2014_multi-realization.nc').rlds)
LWu = (xr.open_mfdataset(file_dir+'rlus_MPI-ESM1-2-HR_abrupt-4xCO21850-2014_multi-realization.nc').rlus)

# latent and sensible heat 
LH = (xr.open_mfdataset(file_dir+'hfls_MPI-ESM1-2-HR_abrupt-4xCO21850-2014_multi-realization.nc').hfls)
SH = (xr.open_mfdataset(file_dir+'hfss_MPI-ESM1-2-HR_abrupt-4xCO21850-2014_multi-realization.nc').hfss)
SWd_clr = (xr.open_mfdataset(file_dir+'rsdscs_MPI-ESM1-2-HR_abrupt-4xCO21850-2014_multi-realization.nc').rsdscs)
LWd_clr = (xr.open_mfdataset(file_dir+'rldscs_MPI-ESM1-2-HR_abrupt-4xCO21850-2014_multi-realization.nc').rldscs)

# surface air temperature  
tas = (xr.open_mfdataset(file_dir+'ts_MPI-ESM1-2-HR_abrupt-4xCO21850-2014_multi-realization.nc').ts)

# dimensions of data
lat = tas.lat
lon = tas.lon
ny = len(lat)
nx = len(lon)
year = tas.year

# heat storage
Q = SWd - SWu + LWd - LWu - LH - SH

# albedo (surface)
alb = SWu/SWd
SWd_cld = SWd - SWd_clr
LWd_cld = LWd - LWd_clr

# climatologies
alb_m = alb.mean(dim='year')
tas_m = tas.mean(dim='year')

tas_m_const = tas.mean(dim='year')

c = 5.670373*1.0e-8

# Regime 1 Cut

SWd = SWd[:,:50]
SWu = SWu[:,:50]
LWd = LWd[:,:50]
LWu = LWu[:,:50]

LH = LH[:,:50]
SH = SH[:,:50]
SWd_clr = SWd_clr[:,:50]
LWd_clr = LWd_clr[:,:50]

tas = tas[:,:50]

Q = Q[:,:50]

alb = alb[:,:50]
SWd_cld = SWd_cld[:,:50]
LWd_cld = LWd_cld[:,:50]

alb_m = alb_m
tas_m = tas_m
print("Regime 1")

# for use in loop:

SWduse = SWd
SWuuse = SWu
LWduse = LWd
LWuuse = LWu

LHuse = LH
SHuse = SH
SWd_clruse = SWd_clr
LWd_clruse = LWd_clr

tasuse = tas

Quse = Q

albuse = alb
SWd_clduse = SWd_cld
LWd_clduse = LWd_cld

alb_muse = alb_m
tas_muse = tas_m

# Calc partial temp changes (PTCs)
def delta(var):
    # Reshape the data to create a 2D array
    data = np.stack([var.values, amoc], axis=-1)
    
    # Perform the polynomial fit along the last dimension (0: var, 1: amoc)
    polyfit_coeffs = np.polyfit(-data[:, 1], data[:, 0], deg=1)
    
    # Return the coefficient for the linear term (slope)
    return polyfit_coeffs[0]

# Bar Plot -- Calculating Regional Mean for SSP245 ################################################################################################################

def nawh(var):
    # to calculate North Atlantic Warming Hole regional [44W-24W, 48N-60N] mean
    # migth re-define NAWH region 
    # var: an xarray [...,lat,lon]
    if (min(lon) < -100): # lon: -180~180
        x_center = -26.5
    else: # lon: 0~360
        x_center = 333.5
    x_range = 11.5
    #gen = 11.25
    y_center = 54.75
    y_range = 6.25
    #gen = 8.75
    return var.where(abs(var.lat - y_center) <= y_range).where(abs(var.lon - x_center ) <= x_range).mean(('lat', 'lon'))


def ols_conf(y,x=amoc):
    # y is some 1-D variable (xarray)
    # x is the time/year
    x = amoc
    mod = sm.OLS(y.values, sm.add_constant(x)).fit()
    return mod.conf_int(0.1)[1,0]-mod.params[1]

con_int = np.zeros(7)
term_nawh = np.zeros(7)

abrupt1_term_storage = np.zeros((1,7))
abrupt1_con_storage = np.zeros((1,7))

term_name = ['SAF','CRF','non-SAF ClrSky SWR','ClrSky LWR↓','heat storage','SH&LH', 't5 + t6']

########## calculate regional mean PTC 

for r in range(0,1):

    amoc = xr.open_dataset('/Volumes/HDDBackup/CMIP6/Updated_AMOCI/AMOCI_abrupt/AMOCI_abrupt_r1.nc')
    amoc = amoc.msftmz[:50]
    amoc = amoc.values / 1e9

    print("current rlzn = {}".format(r+1))
    #assign current rlzn
    const=4*c*nawh(tas_m_const[r])**3
    SWd = SWduse[r]
    SWu = SWuuse[r]
    LWd = LWduse[r]
    LWu = LWuuse[r]

    LH = LHuse[r]
    SH = SHuse[r]
    SWd_clr = SWd_clruse[r]
    LWd_clr = LWd_clruse[r]

    tas = tasuse[r]

    Q = Quse[r]

    alb = albuse[r]
    SWd_cld = SWd_clduse[r]
    LWd_cld = LWd_clduse[r]

    alb_m = alb_muse[r]
    tas_m = tas_muse[r]

    # surface albedo feedback 
    term_nawh[0] = -delta(nawh(alb))*(nawh(SWd.mean(dim='year'))+delta(nawh(SWd)))/const
    con_int[0] = -ols_conf(nawh(alb))*(nawh(SWd.mean(dim='year')))/const # neglect the second higher-order term
    print('done 1')

    # CRF: cloud radiative forcing term 
    term_nawh[1] = (delta(nawh(LWd_cld))+(1-nawh(alb_m))*delta(nawh(SWd_cld)))/const
    con_int[1] = (ols_conf(nawh(LWd_cld))+(1-nawh(alb_m))*ols_conf(nawh(SWd_cld)))/const
    print('done 2')

    # non-SAF-induced change in clear-sky SW
    term_nawh[2] = (1-nawh(alb_m))*delta(nawh(SWd_clr))/const
    con_int[2] = (1-nawh(alb_m))*ols_conf(nawh(SWd_clr))/const
    print('done 3')

    # downward clear-sky LW change
    term_nawh[3] = delta(nawh(LWd_clr))/const
    con_int[3] = ols_conf(nawh(LWd_clr))/const
    print('done 4')

    # heat storage change
    term_nawh[4] = -delta(nawh(Q))/const
    con_int[4] = -ols_conf(nawh(Q))/const
    print('done 5')

    # surface sensible & latent heat change
    term_nawh[5] = -delta(nawh(LH+SH))/const
    con_int[5] = -ols_conf(nawh(LH+SH))/const
    print('done 6')

    term_nawh[6] = term_nawh[5] + term_nawh[4]
    con_int[6] = 0
    print('done 7')

    abrupt1_term_storage[r] = term_nawh
    abrupt1_con_storage[r] = con_int

In [ ]:
# RUN FOR Abrupt R2

file_dir = '[ABRUPT FILE PATH]'

## use open_mfdataset to open multiple files at the same time
# radiative fluxes 

SWd = (xr.open_mfdataset(file_dir+'rsds_MPI-ESM1-2-HR_abrupt-4xCO21850-2014_multi-realization.nc').rsds)
SWu = (xr.open_mfdataset(file_dir+'rsus_MPI-ESM1-2-HR_abrupt-4xCO21850-2014_multi-realization.nc').rsus)
LWd = (xr.open_mfdataset(file_dir+'rlds_MPI-ESM1-2-HR_abrupt-4xCO21850-2014_multi-realization.nc').rlds)
LWu = (xr.open_mfdataset(file_dir+'rlus_MPI-ESM1-2-HR_abrupt-4xCO21850-2014_multi-realization.nc').rlus)

# latent and sensible heat 
LH = (xr.open_mfdataset(file_dir+'hfls_MPI-ESM1-2-HR_abrupt-4xCO21850-2014_multi-realization.nc').hfls)
SH = (xr.open_mfdataset(file_dir+'hfss_MPI-ESM1-2-HR_abrupt-4xCO21850-2014_multi-realization.nc').hfss)
SWd_clr = (xr.open_mfdataset(file_dir+'rsdscs_MPI-ESM1-2-HR_abrupt-4xCO21850-2014_multi-realization.nc').rsdscs)
LWd_clr = (xr.open_mfdataset(file_dir+'rldscs_MPI-ESM1-2-HR_abrupt-4xCO21850-2014_multi-realization.nc').rldscs)

# surface air temperature  
tas = (xr.open_mfdataset(file_dir+'ts_MPI-ESM1-2-HR_abrupt-4xCO21850-2014_multi-realization.nc').ts)

# dimensions of data
lat = tas.lat
lon = tas.lon
ny = len(lat)
nx = len(lon)
year = tas.year

# heat storage
Q = SWd - SWu + LWd - LWu - LH - SH

# albedo (surface)
alb = SWu/SWd
SWd_cld = SWd - SWd_clr
LWd_cld = LWd - LWd_clr

# climatologies
alb_m = alb.mean(dim='year')
tas_m = tas.mean(dim='year')

tas_m_const = tas.mean(dim='year')

c = 5.670373*1.0e-8

# Regime 1 Cut

SWd = SWd[:,50:]
SWu = SWu[:,50:]
LWd = LWd[:,50:]
LWu = LWu[:,50:]

LH = LH[:,50:]
SH = SH[:,50:]
SWd_clr = SWd_clr[:,50:]
LWd_clr = LWd_clr[:,50:]

tas = tas[:,50:]

Q = Q[:,50:]

alb = alb[:,50:]
SWd_cld = SWd_cld[:,50:]
LWd_cld = LWd_cld[:,50:]

alb_m = alb_m
tas_m = tas_m
print("Regime 1")

# for use in loop:

SWduse = SWd
SWuuse = SWu
LWduse = LWd
LWuuse = LWu

LHuse = LH
SHuse = SH
SWd_clruse = SWd_clr
LWd_clruse = LWd_clr

tasuse = tas

Quse = Q

albuse = alb
SWd_clduse = SWd_cld
LWd_clduse = LWd_cld

alb_muse = alb_m
tas_muse = tas_m

# Calc partial temp changes (PTCs)
def delta(var):
    # Reshape the data to create a 2D array
    data = np.stack([var.values, amoc], axis=-1)
    
    # Perform the polynomial fit along the last dimension (0: var, 1: amoc)
    polyfit_coeffs = np.polyfit(-data[:, 1], data[:, 0], deg=1)
    
    # Return the coefficient for the linear term (slope)
    return polyfit_coeffs[0]

# Bar Plot -- Calculating Regional Mean for SSP245 ################################################################################################################

def nawh(var):
    # to calculate North Atlantic Warming Hole regional mean
    # var: an xarray [...,lat,lon]
    if (min(lon) < -100): # lon: -180~180
        x_center = -26.5
    else: # lon: 0~360
        x_center = 333.5
    x_range = 11.5
    #gen = 11.25
    y_center = 54.75
    y_range = 6.25
    #gen = 8.75
    return var.where(abs(var.lat - y_center) <= y_range).where(abs(var.lon - x_center ) <= x_range).mean(('lat', 'lon'))

def ols_conf(y,x=amoc):
    # y is some 1-D variable (xarray)
    # x is the time/year
    x = amoc
    mod = sm.OLS(y.values, sm.add_constant(x)).fit()
    return mod.conf_int(0.1)[1,0]-mod.params[1]

con_int = np.zeros(7)
term_nawh = np.zeros(7)

abrupt2_term_storage = np.zeros((1,7))
abrupt2_con_storage = np.zeros((1,7))

term_name = ['SAF','CRF','non-SAF ClrSky SWR','ClrSky LWR↓','heat storage','SH&LH', 't5 + t6']

########## calculate regional mean PTC 

for r in range(0,1):

    amoc = xr.open_dataset('/Volumes/HDDBackup/CMIP6/Updated_AMOCI/AMOCI_abrupt/AMOCI_abrupt_r1.nc')
    amoc = amoc.msftmz[50:]
    amoc = amoc.values / 1e9

    print("current rlzn = {}".format(r+1))
    #assign current rlzn
    const=4*c*nawh(tas_m_const[r])**3
    SWd = SWduse[r]
    SWu = SWuuse[r]
    LWd = LWduse[r]
    LWu = LWuuse[r]

    LH = LHuse[r]
    SH = SHuse[r]
    SWd_clr = SWd_clruse[r]
    LWd_clr = LWd_clruse[r]

    tas = tasuse[r]

    Q = Quse[r]

    alb = albuse[r]
    SWd_cld = SWd_clduse[r]
    LWd_cld = LWd_clduse[r]

    alb_m = alb_muse[r]
    tas_m = tas_muse[r]

    # surface albedo feedback 
    term_nawh[0] = -delta(nawh(alb))*(nawh(SWd.mean(dim='year'))+delta(nawh(SWd)))/const
    con_int[0] = -ols_conf(nawh(alb))*(nawh(SWd.mean(dim='year')))/const # neglect the second higher-order term
    print('done 1')

    # CRF: cloud radiative forcing term 
    term_nawh[1] = (delta(nawh(LWd_cld))+(1-nawh(alb_m))*delta(nawh(SWd_cld)))/const
    con_int[1] = (ols_conf(nawh(LWd_cld))+(1-nawh(alb_m))*ols_conf(nawh(SWd_cld)))/const
    print('done 2')

    # non-SAF-induced change in clear-sky SW
    term_nawh[2] = (1-nawh(alb_m))*delta(nawh(SWd_clr))/const
    con_int[2] = (1-nawh(alb_m))*ols_conf(nawh(SWd_clr))/const
    print('done 3')

    # downward clear-sky LW change
    term_nawh[3] = delta(nawh(LWd_clr))/const
    con_int[3] = ols_conf(nawh(LWd_clr))/const
    print('done 4')

    # heat storage change
    term_nawh[4] = -delta(nawh(Q))/const
    con_int[4] = -ols_conf(nawh(Q))/const
    print('done 5')

    # surface sensible & latent heat change
    term_nawh[5] = -delta(nawh(LH+SH))/const
    con_int[5] = -ols_conf(nawh(LH+SH))/const
    print('done 6')

    term_nawh[6] = term_nawh[5] + term_nawh[4]
    con_int[6] = 0
    print('done 7')

    abrupt2_term_storage[r] = term_nawh
    abrupt2_con_storage[r] = con_int

In [ ]:
hist_values = np.mean(hist_term_storage, axis=0)
hist_con_values = np.mean(hist_con_storage, axis=0)

ssp_values = np.mean(ssp_term_storage, axis=0)
ssp_con_values = np.mean(ssp_con_storage, axis=0)

abr1_values = np.mean(abrupt1_term_storage, axis=0)
abr1_con_values = np.mean(abrupt1_con_storage, axis=0)

abr2_values = np.mean(abrupt2_term_storage, axis=0)
abr2_con_values = np.mean(abrupt2_con_storage, axis=0)

In [ ]:
###### bar plot

fig = plt.figure(figsize=(16,8))
ax  = fig.add_axes([0.07, 0.14, 0.9, 0.75])
plt.title("Partial Temperature Changes (PTC) per regime", fontdict={'weight': 'bold', 'size': 16})
    
xticks=('T1','T2','T3','T4','T5','T6', 'T5+T6')
x = np.arange(1,8)
width = 0.2 
hist_bar = ax.bar(x-0.3, hist_values, yerr=abs(hist_con_values), width=width, color='c')
abrupt1_bar = ax.bar(x-0.1, abr1_values, yerr=abs(abr1_con_values),width=width, color='r')
abrupt2_bar = ax.bar(x+0.1, abr2_values, yerr=abs(abr2_con_values),width=width, color='orange')
ssp_bar =  ax.bar(x+0.3, ssp_values, yerr=abs(ssp_con_values), width=width, color='g')
plt.axhline(y=0,color='k',linewidth=0.8)
plt.axhline(y=sum(ssp_values[:6]),color='g',linewidth=1.2)
plt.axhline(y=sum(abr1_values[:6]),color='r',linewidth=1.2)
plt.axhline(y=sum(abr2_values[:6]),color='orange',linewidth=1.2)
plt.axhline(y=sum(hist_values[:6]),color='c',linewidth=1.2)
ax.set_ylabel('NAWH-SST change [K/-1 Sv]',fontsize=16)
plt.xticks(x,xticks,rotation=20)
ax.tick_params(labelsize=14)

plt.rcParams.update({'font.family':'sans-serif'})
plt.rcParams.update({'font.sans-serif':'Arial'})
plt.rcParams.update({'font.size': 13})
for label in (ax.get_xticklabels() + ax.get_yticklabels()):
	label.set_fontsize(15)
plt.minorticks_on()
from matplotlib.ticker import MultipleLocator
ax.yaxis.set_minor_locator(MultipleLocator(0.050))
ax.xaxis.set_minor_locator(MultipleLocator(100)) 

ssp_bar.set_label("SSP245")
abrupt1_bar.set_label("Abrupt Regime 1")
hist_bar.set_label("Historical")
abrupt2_bar.set_label("Abrupt Regime 2")
plt.legend(loc=4, frameon=True)

fig_dir = '[OUTPUT PATH]'
fig_name = 'PTC Bar Plot'
plt.savefig(fig_dir + fig_name, dpi = 600, facecolor='white', transparent=False)
plt.show()

t1 = sum(hist_values[:6])
print("hist: ", + t1*100)
t2 = sum(abr1_values[:6])
print("abrupt1:", + t2*100)
t3 = sum(abr2_values[:6])
print("abrupt2:", + t3*100)
t4 = sum(ssp_values[:6])
print("ssp:", + t4*100)